In [1]:
import pandas as pd
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC

C:\Users\qubadlim\AppData\Local\Temp\ipykernel_15220\3805729115.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [11]:
base_url = 'https://metbuat.az/'
driver.get(base_url)

In [14]:
news_links = []

while True:

    news_container = driver.find_element(By.CSS_SELECTOR, '.ml50')
    news_elements = news_container.find_elements(By.CSS_SELECTOR, '.news_box')

    for element in news_elements:
        news_links.append(element.get_attribute('href'))

    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'li.next a')
        driver.execute_script("arguments[0].click();", next_button)

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.next a')))
    except Exception as e:
        print(e)
        break

print(len(news_links));

KeyboardInterrupt: 

In [15]:
print(len(news_links))

6786


In [24]:
def get_news_infos(url):

    driver.get(url)

    news_infos = {}

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.news_in_content')))

    news_container = driver.find_element(By.CSS_SELECTOR, '.news_in_content')
    news_title = news_container.find_element(By.CSS_SELECTOR, '.news_in_ttl')
    news_category = news_container.find_element(By.CSS_SELECTOR, 'span.news_in_catg a')
    news_content_container = news_container.find_element(By.ID, 'maincontent').find_elements(By.TAG_NAME, 'p')

    news_content = ''

    for element in news_content_container:
        news_content += element.text

    news_infos['summary'] = news_title.text
    news_infos['category'] = news_category.text
    news_infos['content'] = news_content

    return news_infos

In [27]:
all_news_infos = []

for link in news_links[:50]:
    news_info = get_news_infos(link)
    all_news_infos.append(news_info)

In [28]:
df = pd.DataFrame(all_news_infos)

In [31]:
df.head()

,summary,category,content
0,181 avtobus yeni nəqliyyat mübadilə mərkəzində...,Ölkə,Bu gündən 12 müntəzəm marşrut xətti üzrə 181 a...
1,“Qarabağ” – “Neftçi” oyununun vaxtı bəlli oldu,İdman,Azərbaycan kubokunda 1/2 final mərhələsinin ca...
2,"""Crocus""a görə 9 nəfər bu ölkədə saxlanıldı",Dünya,Rusiyada “Crocus City Hall” konsert salonunda ...
3,Azərbaycan Premyer Liqasında iki turun təqvimi...,İdman,Azərbaycan Premyer Liqasında iki turun təqvimi...
4,Ağrıkəsici olan ən yaxşı 5 qida,Sağlamlıq,Bəzi bitkilər və meyvələr min illərdir təbii a...


In [35]:
df.to_csv('metbuataz.csv')